In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

In [3]:
#vecorisation of data
input_texts = []
target_texts = []
input_words = set()
target_words = set()
with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text  = line.split('\t')
    target_text = 'START_ ' + target_text + ' _END'
    input_texts.append(input_text)
    target_texts.append(target_text)
    words = input_text.split(' ') 
    for word in words:
        if word not in input_words:
            input_words.add(word)
    words = target_text.split(' ') 
    for word in words:
        if word not in target_words:
            target_words.add(word)
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
input_text_sizes = []
for text in input_texts:
    input_text_sizes.append(len(text.split(' ')))
target_text_sizes = []
for text in target_texts:
    target_text_sizes.append(len(text.split(' ')))
max_encoder_seq_length = max(input_text_sizes)
max_decoder_seq_length = max(target_text_sizes)

In [4]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 20000
Number of unique input tokens: 5306
Number of unique output tokens: 9944
Max sequence length for inputs: 5
Max sequence length for outputs: 14


In [5]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [6]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [7]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, word in enumerate(input_text.split(' ')):
        encoder_input_data[i,t]=input_token_index[word]
    for t, word in enumerate(target_text.split(' ')):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word] 
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [8]:
embedding_size = 50

In [9]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     265300      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     497200      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [11]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=100,
          validation_split=0.15)
model.save('s2s1.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 17000 samples, validate on 3000 samples
Epoch 1/100
17000/17000 [==============================] - 59s 3ms/step - loss: 2.1434 - acc: 0.0711 - val_loss: 2.2468 - val_acc: 0.0714
Epoch 2/100
17000/17000 [==============================] - 64s 4ms/step - loss: 1.8040 - acc: 0.0726 - val_loss: 2.1399 - val_acc: 0.0876
Epoch 3/100
17000/17000 [==============================] - 195s 11ms/step - loss: 1.6771 - acc: 0.0872 - val_loss: 2.0329 - val_acc: 0.0916
Epoch 4/100
17000/17000 [==============================] - 56s 3ms/step - loss: 1.5769 - acc: 0.0923 - val_loss: 1.9563 - val_acc: 0.0975
Epoch 5/100
17000/17000 [==============================] - 56s 3ms/step - loss: 1.5028 - acc: 0.1008 - val_loss: 1.9198 - val_acc: 0.1030
Epoch 6/100
17000/17000 [==============================] - 105s 6ms/step - loss: 1.4471 - acc: 0.1099 - val_loss: 1.8646 - val_acc: 0

17000/17000 [==============================] - 56s 3ms/step - loss: 0.7098 - acc: 0.2208 - val_loss: 1.6692 - val_acc: 0.1651
Epoch 56/100
17000/17000 [==============================] - 56s 3ms/step - loss: 0.7055 - acc: 0.2218 - val_loss: 1.6691 - val_acc: 0.1654
Epoch 57/100
17000/17000 [==============================] - 56s 3ms/step - loss: 0.7017 - acc: 0.2229 - val_loss: 1.6691 - val_acc: 0.1656
Epoch 58/100
17000/17000 [==============================] - 56s 3ms/step - loss: 0.6976 - acc: 0.2230 - val_loss: 1.6759 - val_acc: 0.1656
Epoch 59/100
17000/17000 [==============================] - 56s 3ms/step - loss: 0.6937 - acc: 0.2241 - val_loss: 1.6770 - val_acc: 0.1658
Epoch 60/100
17000/17000 [==============================] - 57s 3ms/step - loss: 0.6901 - acc: 0.2252 - val_loss: 1.6769 - val_acc: 0.1656
Epoch 61/100
17000/17000 [==============================] - 56s 3ms/step - loss: 0.6867 - acc: 0.2260 - val_loss: 1.6782 - val_acc: 0.1668
Epoch 62/100
17000/17000 [==============

/home/purva/anaconda3/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [13]:
encoder_model = Model(encoder_inputs, encoder_states)

In [14]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [15]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [22]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['Go.']
Decoded sentence:  Va ! _END
-
Input sentence: ['Hi.']
Decoded sentence:  Salut ! _END
-
Input sentence: ['Hi.']
Decoded sentence:  Salut ! _END
-
Input sentence: ['Run!']
Decoded sentence:  Fais un homme. _END
-
Input sentence: ['Run!']
Decoded sentence:  Fais un homme. _END
-
Input sentence: ['Who?']
Decoded sentence:  Qui ? _END
-
Input sentence: ['Wow!']
Decoded sentence:  Ça te plaît ? _END
-
Input sentence: ['Fire!']
Decoded sentence:  ! _END
-
Input sentence: ['Help!']
Decoded sentence:  À plus ici ? _END
-
Input sentence: ['Jump.']
Decoded sentence:  Continue ! _END
-
Input sentence: ['Stop!']
Decoded sentence:  Ça es ! _END
-
Input sentence: ['Stop!']
Decoded sentence:  Ça es ! _END
-
Input sentence: ['Stop!']
Decoded sentence:  Ça es ! _END
-
Input sentence: ['Wait!']
Decoded sentence:  Attends ! _END
-
Input sentence: ['Wait!']
Decoded sentence:  Attends ! _END
-
Input sentence: ['Go on.']
Decoded sentence:  Va ! _END
-
Input sentence: ['Go on.']
De